In [1]:
#General packages
import sys
# import json
# import csv
# import numpy as np
import pandas as pd
from tqdm import tqdm
import os
#CPE package
from cpie.pipelines import Comp2Prot
#Make sure these are installed prior to running
import time


# Load in Required Datasets

In [2]:
#Downloaded from BindingDB on 3/30/2023
file_path=os.path.expanduser(os.getcwd()+'\\data\\BindingDB.csv')
BDB_data=pd.read_csv(file_path,sep=',',usecols=['CID', 'Ligand SMILES','Ligand InChI','BindingDB MonomerID','Ligand InChI Key','BindingDB Ligand Name','Target Name Assigned by Curator or DataSource','Target Source Organism According to Curator or DataSource','Ki (nM)','IC50 (nM)','Kd (nM)','EC50 (nM)','pH','Temp (C)','Curation/DataSource','UniProt (SwissProt) Entry Name of Target Chain','UniProt (SwissProt) Primary ID of Target Chain'],on_bad_lines='skip')

#Downloaded from STITCH on 2/22/2023
file_path=os.path.expanduser(os.getcwd()+'\\data\\STITCH.tsv')
sttch_data=pd.read_csv(file_path,sep='\t')

#Downloaded from ChEMBL on 2/01/2024
file_path=os.path.expanduser(os.getcwd()+'\\data\\ChEMBL.csv')
chembl_data=pd.read_csv(file_path,sep=',')

file_path=os.path.expanduser(os.getcwd()+'\\data\\CTD.csv')
CTD_data=pd.read_csv(file_path,sep=',')

#Downloaded from DTC on 2/24/2023
file_path=os.path.expanduser(os.getcwd()+'\\data\\DTC.csv')
DTC_data=pd.read_csv(file_path,sep=',',usecols=['CID', 'compound_id','standard_inchi_key','target_id','gene_names','wildtype_or_mutant','mutation_info','standard_type','standard_relation','standard_value','standard_units','activity_comment','pubmed_id','doc_type'])

#Downloaded from DrugBank on 3/2/2022
# File obtained in the last cells of this notebook
file_path=os.path.expanduser(os.getcwd()+'\\data\\DB.csv')
DB_data=pd.read_csv(file_path, sep=',')

#Downloaded from DrugCentral on 2/25/2024
# File obtained in the last cells of this notebook
file_path=os.path.expanduser(os.getcwd()+'\\data\\DrugCentral.csv')
DC_data=pd.read_csv(file_path, sep=',')


C:\Users\andre\AppData\Local\Temp\ipykernel_25568\4108575009.py:3: DtypeWarning: Columns (8,9,10,11,15,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  BDB_data=pd.read_csv(file_path,sep=',',usecols=['CID', 'Ligand SMILES','Ligand InChI','BindingDB MonomerID','Ligand InChI Key','BindingDB Ligand Name','Target Name Assigned by Curator or DataSource','Target Source Organism According to Curator or DataSource','Ki (nM)','IC50 (nM)','Kd (nM)','EC50 (nM)','pH','Temp (C)','Curation/DataSource','UniProt (SwissProt) Entry Name of Target Chain','UniProt (SwissProt) Primary ID of Target Chain'],on_bad_lines='skip')
C:\Users\andre\AppData\Local\Temp\ipykernel_25568\4108575009.py:11: DtypeWarning: Columns (27,28,30,31,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  chembl_data=pd.read_csv(file_path,sep=',')
C:\Users\andre\AppData\Local\Temp\ipykernel_25568\4108575009.py:18: DtypeWarning: Columns (0,1,8,14) have mixed types. Speci

In [3]:
dbs = {
    'chembl': chembl_data,
    'bdb': BDB_data,
    'stitch': sttch_data,
    'ctd': CTD_data,
    'dtc': DTC_data,
    'db': DB_data,
    'dc': DC_data
}

# Load in Compound List

In [4]:
chem_dat=pd.read_csv('data/input/PubChem_NicotineMetabolites.csv')

In [5]:
chem_dat

,cid,cmpdname,cmpdsynonym,mw,mf,polararea,complexity,xlogp,heavycnt,hbonddonor,...,gpfamilycnt,neighbortype,meshheadings,annothits,annothitcnt,aids,cidcdate,sidsrcname,depcatg,annotation
0,89594,Nicotine,nicotine|L-Nicotine|54-11-5|(-)-Nicotine|(S)-N...,162.23,C10H14N2,16.1,147.0,1.2,12,0,...,37613,2D+3D,Nicotine,Agrochemical Information|Biological Test Resul...,16,421|426|427|433|434|435|445|530|540|541|542|54...,20040916,001Chemical|10X CHEM|1st Scientific|3B Scienti...,Chemical Vendors|Curation Efforts|Governmental...,N - Nervous system > N07 - Other nervous syste...
1,854019,Cotinine,cotinine|(-)-Cotinine|486-56-6|(S)-Cotinine|Co...,176.21,C10H12N2O,33.2,205.0,-0.3,13,0,...,2008,2D+3D,Cotinine,Biological Test Results|Chemical and Physical ...,15,377|422|429|436|547|608|694|732|940|1033|1332|...,20050709,001Chemical|10X CHEM|3B Scientific (Wuhan) Cor...,Chemical Vendors|Curation Efforts|Governmental...,C78272 - Agent Affecting Nervous System > C477...
2,107963,Hydroxycotinine,Hydroxycotinine|trans-3'-hydroxycotinine|34834...,192.21,C10H12N2O2,53.4,232.0,-0.9,14,1,...,60,2D+3D,NaN,Biological Test Results|Classification|Literat...,9,540209|540210|540212|540213,20050808,10X CHEM|A2B Chem|AA BLOCKS|AbaChemScene|ABI C...,Chemical Vendors|Curation Efforts|Governmental...,NaN
3,413,Norcotinine,"norcotinine|(R,S)-Norcotinine|17708-87-1|5-(PY...",162.19,C9H10N2O,42.0,181.0,0.1,12,1,...,82,2D+3D,NaN,Biological Test Results|Chemical and Physical ...,11,1076764|1076765|1076766|1076767|1076768|107676...,20050623,001Chemical|3WAY PHARM INC|A2B Chem|AA BLOCKS|...,Chemical Vendors|Curation Efforts|Governmental...,NaN
4,9815514,cotinine N-oxide,cotinine N-oxide|36508-80-2|(S)-Cotinine N-Oxi...,192.21,C10H12N2O2,45.8,232.0,-1.3,14,0,...,0,2D+3D,NaN,Chemical and Physical Properties|Classificatio...,9,NaN,20061025,001Chemical|Alfa Chemistry|Ambinter|Aurora Fin...,Chemical Vendors|Curation Efforts|Governmental...,NaN
5,416,3-Hydroxycotinine glucuronide,3-Hydroxycotinine glucuronide|Trans-3'-Hydroxy...,368.34,C16H20N2O8,150.0,546.0,-1.7,26,4,...,0,2D,NaN,Classification|Literature|Pharmacology and Bio...,3,NaN,20050623,AAA Chemistry|ABI Chem|Biocore|ChemDB|Cheminfo...,Chemical Vendors|Curation Efforts|Legacy Depos...,NaN
6,168719985,2-Hydroxyfluorene sulfate,2-hydroxyfluorene sulfate,276.27,C13H8O5S,81.2,470.0,2.7,19,1,...,0,NaN,NaN,Literature,1,NaN,20231107,Springer Nature,Journal Publishers,NaN


# Run CPE on Compound List found within PubChem

In [6]:
cpe = Comp2Prot('local', dbs=dbs)

In [7]:
times = {}

In [23]:
cpe_dat=pd.DataFrame()
cpe_fail=pd.DataFrame()
states = pd.DataFrame()

r=0
for h in tqdm(range(0, len(chem_dat))):

    try:
        inputid=int(chem_dat[' cid'].iloc[h])
    except:
        continue
    start = time.time()

    # try:
    [comp_dat, state] = cpe.comp_interactions(inputid)
    end = time.time()
    times[h] = end-start
    states = pd.concat([states, state])
    cpe_dat=pd.concat([cpe_dat,comp_dat])
    print(f'{h} completed')
    # except:
    #     print(f'{h} failed')
    #     cpe_fail.loc[r,"failed_id"]=inputid #Collects failed inputids
    #     r=r+1

    # Saves a file after a certain number of loops for insurance
    # check=h/50 #automatically saves progress when on loop number set here
    # if (check-int(check)==0)==True:
    #     filename='results/gut/cpe_iter_'+str(h)+'.csv'
    #     cpe_dat.to_csv(filename,sep=',',index=False)


  0%|          | 0/7 [00:00<?, ?it/s]c:\Users\andre\Desktop\compound-protein-interaction\network_medicine\pipelines\CompProtExtract.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tar_all = pd.concat([tar_all, result])


pc done!
chembl done!
bdb done!
stitch done!
ctd done!
dtc done!
otp done!
dc done!
db done!


c:\Users\andre\Desktop\compound-protein-interaction\network_medicine\pipelines\Pipeline.py:78: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Sources do not provide activity data' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.loc[index,'std_pchembl'] = 'Sources do not provide activity data'
 14%|█▍        | 1/7 [00:16<01:39, 16.62s/it]

0 completed
pc done!
chembl done!


c:\Users\andre\Desktop\compound-protein-interaction\network_medicine\pipelines\CompProtExtract.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tar_all = pd.concat([tar_all, result])


...

otp done!
dc done!
db done!
4 completed
pc done!
chembl done!
bdb done!
stitch done!
ctd done!
dtc done!


 86%|████████▌ | 6/7 [00:47<00:06,  6.34s/it]

otp done!
dc done!
db done!
5 completed
pc done!
chembl done!
bdb done!


100%|██████████| 7/7 [00:51<00:00,  7.33s/it]

stitch done!
ctd done!
dtc done!
otp done!
dc done!
db done!
6 completed


In [25]:
cpe_dat

,input_id,inchi,inchikey,isomeric_smiles,iupac_name,entrez,gene_type,hgnc_symbol,description,pchembl_count,...,src_count,pubchem,chembl,bindingdb,stitch,ctd,dtc,otp,drugcentral,drugbank
0,89594.0,InChI=1S/C10H14N2/c1-12-7-3-5-10(12)9-4-2-6-11...,SNICXCGAKADSCV-JTQLQIEISA-N,CN1CCC[C@H]1C2=CN=CC=C2,3-[(2S)-1-methylpyrrolidin-2-yl]pyridine,1143,protein_coding,CHRNB4,cholinergic receptor nicotinic beta 4 subunit ...,20,...,4,1,0,0,1,1,0,0,0,1
1,89594.0,InChI=1S/C10H14N2/c1-12-7-3-5-10(12)9-4-2-6-11...,SNICXCGAKADSCV-JTQLQIEISA-N,CN1CCC[C@H]1C2=CN=CC=C2,3-[(2S)-1-methylpyrrolidin-2-yl]pyridine,1134,protein_coding,CHRNA1,cholinergic receptor nicotinic alpha 1 subunit...,6,...,4,1,0,1,1,0,1,0,0,0
2,89594.0,InChI=1S/C10H14N2/c1-12-7-3-5-10(12)9-4-2-6-11...,SNICXCGAKADSCV-JTQLQIEISA-N,CN1CCC[C@H]1C2=CN=CC=C2,3-[(2S)-1-methylpyrrolidin-2-yl]pyridine,1141,protein_coding,CHRNB2,cholinergic receptor nicotinic beta 2 subunit ...,45,...,6,1,0,0,1,1,0,1,1,1
3,89594.0,InChI=1S/C10H14N2/c1-12-7-3-5-10(12)9-4-2-6-11...,SNICXCGAKADSCV-JTQLQIEISA-N,CN1CCC[C@H]1C2=CN=CC=C2,3-[(2S)-1-methylpyrrolidin-2-yl]pyridine,1137,protein_coding,CHRNA4,cholinergic receptor nicotinic alpha 4 subunit...,44,...,7,1,0,1,1,1,1,0,1,1
4,89594.0,InChI=1S/C10H14N2/c1-12-7-3-5-10(12)9-4-2-6-11...,SNICXCGAKADSCV-JTQLQIEISA-N,CN1CCC[C@H]1C2=CN=CC=C2,3-[(2S)-1-methylpyrrolidin-2-yl]pyridine,1144,protein_coding,CHRND,cholinergic receptor nicotinic delta subunit [...,3,...,2,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,89594.0,InChI=1S/C10H14N2/c1-12-7-3-5-10(12)9-4-2-6-11...,SNICXCGAKADSCV-JTQLQIEISA-N,CN1CCC[C@H]1C2=CN=CC=C2,3-[(2S)-1-methylpyrrolidin-2-yl]pyridine,6584,protein_coding,SLC22A5,solute carrier family 22 member 5 [Source:HGNC...,0,...,1,0,0,0,0,0,0,0,0,1
73,89594.0,InChI=1S/C10H14N2/c1-12-7-3-5-10(12)9-4-2-6-11...,SNICXCGAKADSCV-JTQLQIEISA-N,CN1CCC[C@H]1C2=CN=CC=C2,3-[(2S)-1-methylpyrrolidin-2-yl]pyridine,6583,protein_coding,SLC22A4,solute carrier family 22 member 4 [Source:HGNC...,0,...,1,0,0,0,0,0,0,0,0,1
0,854019.0,InChI=1S/C10H12N2O/c1-12-9(4-5-10(12)13)8-3-2-...,UIKROCXWUNQSPJ-VIFPVBQESA-N,CN1[C@@H](CCC1=O)C2=CN=CC=C2,(5S)-1-methyl-5-pyridin-3-ylpyrrolidin-2-one,1548,protein_coding,CYP2A6,cytochrome P450 family 2 subfamily A member 6 ...,3,...,2,0,0,1,0,0,1,0,0,0
1,854019.0,InChI=1S/C10H12N2O/c1-12-9(4-5-10(12)13)8-3-2-...,UIKROCXWUNQSPJ-VIFPVBQESA-N,CN1[C@@H](CCC1=O)C2=CN=CC=C2,(5S)-1-methyl-5-pyridin-3-ylpyrrolidin-2-one,1544,protein_coding,CYP1A2,cytochrome P450 family 1 subfamily A member 2 ...,1,...,2,0,0,1,0,0,1,0,0,0


In [28]:
syns = chem_dat[[' cid', 'cmpdsynonym']].rename(columns={' cid': 'input_id', 'cmpdsynonym': 'synonyms'})

In [30]:
cpe_dat = cpe_dat.merge(syns, on='input_id', how='left')

In [32]:
cpe_dat = cpe_dat[['input_id', 'inchi', 'inchikey', 'isomeric_smiles', 'iupac_name',
       'synonyms', 'entrez', 'gene_type', 'hgnc_symbol', 'description', 'pchembl_count',
       'ave_pchembl', 'std_pchembl', 'src_count', 'pubchem', 'chembl',
       'bindingdb', 'stitch', 'ctd', 'dtc', 'otp', 'drugcentral', 'drugbank']]

In [36]:
cpe_dat.to_csv('NicotineMetabolites.csv', index=False)

In [37]:
states.to_csv('NicotineMetabolites_states.csv', index=False)

In [ ]:
cpe_dat

# Run CPE on Single Compound

## CPE with all Databases

### Test time required

In [ ]:
#inputid='AAWZDTNXLSGCEK-WYWMIBKRSA-N'
import cProfile
inputid=5350

In [ ]:
cProfile.run('cpe.comp_interactions(inputid)')

### Run pipeline

In [ ]:
inputid = 317

In [ ]:
cpe = Comp2Prot('local', dbs=dbs)

In [ ]:
[comp_dat,states]=cpe.comp_interactions(input_id=inputid)

In [ ]:
states

In [ ]:
comp_dat

### CPE with select Databases

In [ ]:
cpe = Comp2Prot('local', dbs=dbs)

In [ ]:
#inputid='AAWZDTNXLSGCEK-WYWMIBKRSA-N'
inputid=39929

In [ ]:
#The second argument is a string listing the databases needed.
#Underscore is required to separate the databases.
[comp_dat,states]=cpe.comp_interactions_select(inputid,'ctd')

In [ ]:
comp_dat

# Run each step/database separately (for troubleshooting)

In [ ]:
#inp='AAWZDTNXLSGCEK-WYWMIBKRSA-N'
import pandas as pd
import os
from cpie.utils.identifiers import *
inp=2830389
comp_ids=compound_identifiers(inp)

### Run PubChem only

In [ ]:
from cpie.databases.PubChem import PubChem

In [ ]:
pubchem = PubChem()
pc_dat, pc_state, pc_raw = pubchem.interactions(input_comp=comp_ids, pChEMBL_thres=0)

In [ ]:
pc_dat

In [ ]:
pc_dat

In [ ]:
pc_state

### Run ChEMBL only

In [ ]:
from cpie.databases.ChEMBL import ChEMBL
chembl_ids = []

In [ ]:
file_path=os.path.expanduser(os.getcwd()+'data\\ChEMBL.csv')
chembl_data=pd.read_csv(file_path,sep=',')

In [ ]:
chembl = ChEMBL(database=chembl_data)
[chembl_dat, chembl_state, chembl_raw] = chembl.interactions(input_comp=comp_ids, pChEMBL_thres=0, chembl_ids=chembl_ids)

In [ ]:
chembl_dat

In [ ]:
chembl_state

### Run BindingDB only

In [ ]:
#Downloaded from BindingDB on 3/30/2023
file_path=os.path.expanduser(os.getcwd()+'\\data\\BindingDB_All.tsv')
BDB_data=pd.read_csv(file_path,sep='\t',usecols=['Ligand SMILES','Ligand InChI','BindingDB MonomerID','Ligand InChI Key','BindingDB Ligand Name','Target Name Assigned by Curator or DataSource','Target Source Organism According to Curator or DataSource','Ki (nM)','IC50 (nM)','Kd (nM)','EC50 (nM)','pH','Temp (C)','Curation/DataSource','UniProt (SwissProt) Entry Name of Target Chain','UniProt (SwissProt) Primary ID of Target Chain'],on_bad_lines='skip')

In [ ]:
from cpie.databases.BindingDB import BindingDB

In [ ]:
bdb = BindingDB(database=BDB_data)
[bdb_dat, bdb_state, bdb_raw] = bdb.interactions(input_comp=comp_ids, pChEMBL_thres=0)

In [ ]:
bdb_raw

In [ ]:
bdb_state

In [ ]:
bdb_dat

### Run STITCH only

In [ ]:
#Downloaded from STITCH on 2/22/2023
file_path=os.path.expanduser(os.getcwd()+'\\data\\STITCH.tsv')
sttch_data=pd.read_csv(file_path,sep='\t')

In [ ]:
from cpie.databases.Stitch import Stitch

In [ ]:
stitch = Stitch(database=sttch_data)

In [ ]:
[sttch_dat, sttch_state, sttch_raw] = stitch.interactions(input_comp=comp_ids, set_stereo=1)

In [ ]:
sttch_dat

### Run CTD only

In [ ]:
#Downloaded from CTD on 2/24/2023
file_path=os.path.expanduser(os.getcwd()+'\\data\\CTD.csv')
CTD_data=pd.read_csv(file_path,sep=',')

In [ ]:
from cpie.databases.CTD import CTD

In [ ]:
ctd = CTD(database=CTD_data)
[ctd_dat, ctd_state, ctd_raw] = ctd.interactions(input_comp=comp_ids)

In [ ]:
ctd_dat

### Run DTC only

In [ ]:
#Downloaded from DTC on 2/24/2023
file_path=os.path.expanduser(os.getcwd()+'\\data\\DTC.csv')
DTC_data=pd.read_csv(file_path,sep=',',usecols=['compound_id','standard_inchi_key','target_id','gene_names','wildtype_or_mutant','mutation_info','standard_type','standard_relation','standard_value','standard_units','activity_comment','pubmed_id','doc_type'])

In [ ]:
from cpie.databases.DTC import DTC

In [ ]:
dtc = DTC(database=DTC_data)
[dtc_dat, dtc_state, dtc_raw] = dtc.interactions(input_comp=comp_ids, chembl_ids=[], pChEMBL_thres=0)

In [ ]:
dtc_dat

### Run OTP only

In [ ]:
from cpie.databases.OTP import OTP

In [ ]:
otp = OTP()
[otp_dat, otp_state, otp_raw] = otp.interactions(input_comp=comp_ids, chembl_ids=[], intqual=2)

In [ ]:
otp_dat

### Run DrugCentral only

In [ ]:
# File obtained in the last cells of this notebook
file_path=os.path.expanduser(os.getcwd()+'\\data\\DrugCentral.csv')
DC_data=pd.read_csv(file_path, sep=',')

In [ ]:
from cpie.databases.DrugCentral import DrugCentral

In [ ]:
dc = DrugCentral(database=DC_data)
[DC_dat, DC_state, DC_raw] = dc.interactions(input_comp=comp_ids)

In [ ]:
DC_dat

In [ ]:
DC_state

### Run DrugBank only

In [ ]:
#Downloaded from DrugBank on 3/2/2022
file_path=os.path.expanduser(os.getcwd()+'\\data\\DB.csv')
DB_data = pd.read_csv(file_path, sep=',')

In [ ]:
from cpie.databases.DrugBank import DB

In [ ]:
db = DB(database=DB_data)
[DB_dat, DB_state, DB_raw] = db.interactions(input_comp=comp_ids)

In [ ]:
DB_dat

In [ ]:
DB_state